In [1]:
train_data_path = "./combinedTrainCleaned.json"

In [2]:
import json
train_data = open(train_data_path)
train_data = json.load(train_data)

In [159]:
def sentiment_max_vote(lst):
    sents = {
        "Positive":0,
        "Neutral":0,
        "Negative":0
    }
    
    # loop over all opinions!
    for obj in lst:
        if obj['Polarity'] == 'Positive':
            sents['Positive'] += 1
        if obj['Polarity'] == 'Neutral':
            sents['Neutral'] += 1
        if obj['Polarity'] == 'Negative':
            sents['Negative'] += 1

    ## return 'Neutral' if 'Positive' votes are equal to 'Negative' votes
    if(sents['Positive'] == sents['Negative']):
        return 'Neutral'
    
    ## return sentiment with majority of votes
    return max(sents, key=sents.get)


In [160]:
import random
# function returns two random text samples to use them as GPT-J prompt
def get_two_random_samples(filename):
    # load `filename` contents
    x = open(filename)
    x = json.load(x)
    
    # get two random samples from `filename`
    s1, s2 = random.sample(range(0, len(x)), 2)
    
    # get text labels
    label1 = sentiment_max_vote(x[s1]['opinions'])
    label2 = sentiment_max_vote(x[s2]['opinions'])
    
    # return text1, label1, text2, label2
    return x[s1]['text'], label1, x[s2]['text'], label2

In [163]:
for i in range(0,50):
    print(get_two_random_samples(train_data_path))

("The staff 's foreign languages and general competences were excellent .", 'Positive', 'Good budget option on strip , bit rundown .', 'Neutral')
('And this month , a task force on the project \'s budget concluded that NASA \'s cost estimate for finishing the " core " station between 2002 and 2008 " is not credible . "', 'Neutral', 'Compared with others with whom I am acquainted who earned their doctoral degrees from brick-and-mortars , the education I received was vastly superior .', 'Positive')
('Weekends are little costlier .', 'Negative', 'Ofitsyanty while working well .', 'Positive')
('I just can recommend this hotel to everybody and you should eat at the Asian · Restaurant ... thats just a dream ...', 'Positive', "But after several days of four - and five - hour shifts inside the camp , hearing the prisoners ' requests , he said exchanges have not dipped that deeply into the doctrinal .", 'Neutral')
("Although I wouldn 't say this was a cheap holiday , it didn 't break the bank e

("This was one of the first few mega resorts on the strip in the 90 's , but now it is really starting to show its age .", 'Negative', 'Stayed at the Ritz · Carlton South Beach due to a conference and the experience was excellent .', 'Positive')
("Still , the overseers of the prison are concerned that detainees are n't getting enough pita bread with their meals , and they 're planning to make the food spicier , just the way the prisoners like it back home .", 'Neutral', 'Despite everything , there is still hope in Zimbabwe that this election may bring change .', 'Positive')
('The staff made us feel very welcome from the time we checked in ... to our departure .', 'Positive', 'The US had repeatedly asked Chavez and the Venezuelan government to increase the level of their oil output as well as exports .', 'Positive')
('Devry is very expensive , one of the most expensive schools of its kind .', 'Negative', 'Whether a country agrees with and regardless of how it views the US policy , the U

In [10]:
def get_prompt(text1, label1, text2, label2):
    # define a function that takes as input two samples and generates the prompt
    # that we should pass to the GPT-3 language model for completion.
    description = "Each item in the following list contains a tweet and the respective sentiment. Sentiment is one of 'Positive', 'Negative', or 'Neutral'."
    prompt = (f"{description}\n"
            f"Tweet: {text1} (Sentiment: {label1})\n"
            f"Tweet: {text2} (Sentiment: {label2})\n"
            f"Tweet:")
    return prompt

In [11]:
a,b,c,d = get_two_random_samples(train_data_path);
print(get_prompt(a,b,c,d))

Each item in the following list contains a tweet and the respective sentiment. Sentiment is one of 'Positive', 'Negative', or 'Neutral'.
Tweet: The test , which was carried out in conformity with the rules for such cases , raised suspicions of BSE , which were later confirmed . (Sentiment: Negative)
Tweet: Analysts have warned that however tough Robinson 's talk , international attention is probably too firmly focused on the anti-terror coalition , of which China is a part , to place much pressure on Beijing over human rights . (Sentiment: Negative)
Tweet:


In [12]:
!pip install -q transformers==4.12.5 datasets==1.16.1 tokenizers==0.10.3 openai==0.11.3 requests==2.23.0 sentencepiece==0.1.96 
!pip install pandas==1.3.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 1.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.2
    Uninstalling pandas-1.5.2:
      Successfully uninstalled pandas-1.5.2


In [13]:
from datasets import load_dataset, concatenate_datasets, load_from_disk, load_metric, Dataset, ClassLabel
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, TrainerCallback, AutoModelForCausalLM, AutoModelForSeq2SeqLM

In [16]:
max_input_len = 30
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")

Downloading:   0%|          | 0.00/22.5G [00:00<?, ?B/s]

In [17]:
print(model)

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=

In [22]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=

In [25]:
# define the number of synthetic samples to generate
n = 10
new_texts = []
new_labels = []

iter = 0
while iter < n:
    # select two random samples from training set
    text1, label1, text2, label2 = get_two_random_samples(train_data_path)
    # create the prompt
    prompt = get_prompt(text1, label1, text2, label2)

    # generate text using GPT-J model
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    print('before_gentokens')
    gen_tokens = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=150,)
    print('after_gentokens')
    gen_text = tokenizer.batch_decode(gen_tokens)[0]
    # the generated output will be in the form "<text> (Sentiment: <label>)"
    data = gen_text.split('\n')[3].strip('Tweet: ').split('(Sentiment:')
    if len(data) < 2:
        # the format of the response is invalid
        continue

    text = data[0]
    label = data[1].split(')')[0].strip()
    if label not in ['Positive', 'Negative', 'Neutral']:
        # the format of the response is invalid
        continue
    print('text', text);
    new_texts.append(text)
    new_labels.append(label)
    iter += 1

# define the synthetic dataset and save it to disk 
synthetic_ds = Dataset.from_dict({'text': new_texts, 'label': new_labels})
synthetic_ds.save_to_disk('./gpt-j-generated-data/' + str(n))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


before_gentokens
after_gentokens


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


before_gentokens
after_gentokens
text A nice hotel but... 


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


before_gentokens


KeyboardInterrupt: 

In [26]:
new_texts

['A nice hotel but... ']

In [27]:
new_labels

['Neutral']